# step46, Optimizer로 수행하는 매개변수 갱신

지금까지 경사하강법으로 매개변수로 갱신하였다.  
딥러닝 분야에서는 경사하강법 외에도 다양한 최적화 기법이 제안되고 있다.

이번 단계에서는 매개변수 갱신 작업(갱신 코드)를 모듈화하고 쉽게 다른 모듈로 대체할 수 있는 구조를 만들어 본다.

## 46.1 Optimizer 클래스 

매개변수 갱신을 위한 기반 클래스인 Optimizer를 구현한다.  
Optimizer가 기초를 제공하고,  
구체적인 최적화 기법은 Optimizer 클래스를 상속한 곳에서 구현한다.

In [2]:
# dezero/optimizers.py 
class Optimizer:
    def __init__(self):
        self.target = None 
        self.hooks = []

    def setup(self, target):
        self.target = target
        return self 
    
    def update(self):
        # None 이외에 매개변수를 리스트에 모아둠 
        params = [p for p in self.target.params() if p.grad is not None]

        # 전처리(옵션)
        for f in self.hooks:
            f(params)
        
        # 매개변수 갱신 
        for param in params:
            self.update_one(param)
    
    def update_one(self, param):
        raise NotImplementedError()
    
    def add_hook(self, f):
        self.hooks.append(f)

Optimizer 클래스의 초기화 메서드에서는 target과 hooks라는 두 개의 인스턴스 변수를 초기화  

setup 메서드는 매개변수를 갖는 클래스(Model 또는 Layer)를 인스턴스 변수인 target으로 설정  

update 메서드는 모든 매개변수를 갱신, 인스턴스 변수 grad가 None인 매개변수는 갱신을 건너뜀  

update_one 메서드에서 구체적인 매개변수 갱신을 수행, 이 메서드를 Optimizer의 자식 클래스에서 재정의해야 한다.

Optimizer 클래스는 전체 매개변수를 전처리해주는 기능도 갖췄다.  
원하는 전처리가 있다면 ad_hook 메서들르 사용하여 전처리를 수행하는 함수를 추가한다.  
이 구조 덕에 가중치 감소(Weight Decay)와 기울기 클리핑(Gradient Clipping)같은 기법을 이용할 수 있다.

## 46.2 SGD 클래스 구현

SGD : Stochastic Gradient Descent, 확률적 경사 하강법  

'확률적' : 대상 데이터 중에서 확률적으로 선별한 데이터에 대해 경사하강법을 수행한다는 뜻

딥러닝에서는 원래의 데이터에서 무작위로 골라 경사하강법을 수행하는것이 일반적

In [3]:
# dezero/optimizers.py 

class SGD(Optimizer):
    def __init__(self, lr=0.01):
        super().__init__()
        self.lr = lr 
    
    def update_one(self, param):
        param.data -= self.lr * param.grad.data 

SGD 클래스는 Optimizer 클래스를 상속  
\__init\__ 메서드는 학습률을 받아 초기화  
update_one 메서드에서 매개변수 갱신 코드를 구현

내생각 : 여기서 SGD가 거창한게 아니라 그냥 경사하강법을 Optimizer 클래스를 상속받어 구현할려고 예시처럼 넣어둔거 같다.  
이후 나오는 다른 최적화 기법을 자세히 보면 되겠다.

## 46.3 SGD 클래스를 사용한 문제 해결 

In [4]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np 
from dezero import Variable
from dezero import optimizers
import dezero.functions as F 
from dezero.models import MLP 

np.random.seed(0)
x = np.random.rand(100,1)
y = np.sin(2 * np.pi * x) + np.random.rand(100,1)

lr = 0.2 
max_iter = 10000 
hidden_size = 10 

model = MLP((hidden_size, 1))
optimizer = optimizers.SGD(lr)
optimizer.setup(model)

for i in range(max_iter):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward()

    optimizer.update 

    if i % 1000 == 0:
        print(loss)

variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)
variable(0.8165178492839196)


## 46.4 SGD 이외의 최적화 기법 

Momentum, AdaGrad, AdaDelta, Adam등  
Optimizer 클래스를 도입한 첫번째 목표는 다양한 최적화 기법을 필요에 따라 손쉽게 전환하기 위해서  
그래서 기반 클래스인 Optimizer를 상속하여 다양한 최적화 기법을 구현해본다

Momentum 기법의 수식 표현 

$$v \leftarrow \alpha v - \eta\frac{\partial L}{\partial W}, \qquad \qquad \qquad \qquad (1)$$  

$$W \leftarrow W + v, \qquad \qquad \qquad \qquad  \qquad (2)$$

$W$는 갱신할 가중치 매개변수  
$\frac{\partial L}{\partial W}$는 기울기($W$에 관한 손실 함수 $L$의 기울기)  
$\eta$는 학습률을 뜻함  
$v$는 물리에서 말하는 '속도'에 해당
$\alpha v$ 항은 물체가 아무런 힘을 받지 않을 떄 서서히 감속시키는 역할

(1)은 물체가 기울기 방향으로 힘을 받아 가속되는 물리 법칙을 나타냄  
(2)는 속도만큼 위치(매개변수)가 이동


In [5]:
# dezero/optimizers.py 
import numpy as np 

class MomentumSGD(Optimizer):
    def __init__(self, lr=0.01, momentum=0.9):
        super().__init__()
        self.lr = lr 
        self.momentum = momentum
        self.vs = {}
    
    def update_one(self, param):
        v_key = id(param)
        if v_key not in self.vs:
            self.vs[v_key] = np.zeros_like(param.data)
        
        v = self.vs[v_key]
        v *= self.momentum
        v -= self.lr * param.grad.data 
        param.data += v

각 매개변수에는 '속도'에 해당하는 데이터가 있다. 이 데이터들을 딕셔너리 타입의 인스턴스 변수 self.vs에 유지  
초기화 하는 사이에는 vs에 아무것도 담겨있지 않다.  
update_one()이 호출될때 매개변수와 같은 타입의 데이터를 생성  
그다음은 (1)과 (2)

이제 손쉽게 Momentum으로 전환할 수 있다.  
optimizer = SGD(lr)을 optimizer = MomentumSGD(lr)  
로 바꿔주기만 하면 된다

optimizers.py에는 더 여러 최적화 기법들이 있다.

In [6]:
# dezero/optimizers.py 
class Adam(Optimizer):
    def __init__(self, alpha=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        super().__init__()
        self.t = 0
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.ms = {}
        self.vs = {}

    def update(self, *args, **kwargs):
        self.t += 1
        super().update(*args, **kwargs)

    @property
    def lr(self):
        fix1 = 1. - math.pow(self.beta1, self.t)
        fix2 = 1. - math.pow(self.beta2, self.t)
        return self.alpha * math.sqrt(fix2) / fix1

    def update_one(self, param):
        xp = cuda.get_array_module(param.data)

        key = id(param)
        if key not in self.ms:
            self.ms[key] = xp.zeros_like(param.data)
            self.vs[key] = xp.zeros_like(param.data)

        m, v = self.ms[key], self.vs[key]
        beta1, beta2, eps = self.beta1, self.beta2, self.eps
        grad = param.grad.data

        m += (1 - beta1) * (grad - m)
        v += (1 - beta2) * (grad * grad - v)
        param.data -= self.lr * m / (xp.sqrt(v) + eps)